<a href="https://colab.research.google.com/github/alemano/accidentalidad-tulua/blob/main/notebooks/etl_accidentalidad_tulua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Te damos la bienvenida a Colab

ANALISIS EXPLORATORIO DE DATOS

In [26]:
## IMPORTE DE BIBLIOTECAS
import pandas as pd
import matplotlib.pyplot as plt


In [27]:
## declaro la ruta **** pilas utilizar raw para archivos de git
url = "https://raw.githubusercontent.com/alemano/accidentalidad-tulua/main/01%20data/accidentalidad_tulua_raw.csv"
df = pd.read_csv(url)
df.head()


,AÑO,FECHA,DIA,HORA,AREA,DIRECCION HECHO,CONTROLES DE TRANSITO,BARRIO HECHO,CLASE DE ACCIDENTE,CLASE DE SERVICIO,GRAVEDAD DEL ACCIDENTE,CLASE DE VEHICULO,Cordenada Geografica
0,"2,023",2023/01/03,martes,15:40:00,URBANA,CARRERA 21 CALLE 32,VERTICAL,SAJONIA,CHOQUE,PARTICULAR,CON HERIDOS,MOTOCICLETA,POINT (-76.201795 4.080615)
1,"2,023",2023/01/01,domingo,03:30:00,URBANA,CARRERA 25 CALLE 31-19,NO INFORMA,No informa,CHOQUE,PARTICULAR,CON MUERTO,MOTOCICLETA,POINT (-76.204669 4.082734)
2,"2,023",2023/01/02,lunes,17:30:00,URBANA,CARERRA 40 CALLE 13C,NO INFORMA,No informa,CHOQUE,NO INFORMA,CON HERIDOS,CAMION,POINT (-76.191499 4.089869)
3,"2,023",2023/01/04,miercoles,09:53:00,RURAL,LA IBERIA,NO INFORMA,LA MARINA,CHOQUE,PARTICULAR,CON MUERTO,MOTOCICLETA,POINT (-76.108301 4.0635)
4,"2,023",2023/01/06,viernes,10:09:00,RURAL,VIA FENICIA VEREDA LAS PALMAS,NO INFORMA,LAS PALMAS,VOLCAMIENTO,PARTICULAR,CON MUERTO,AUTOMOVIL,POINT (-76.191499 4.089869)


In [8]:
## tamaño del dataset filas - columnas
df.shape

(869, 13)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   AÑO                     869 non-null    object
 1   FECHA                   869 non-null    object
 2   DIA                     869 non-null    object
 3   HORA                    869 non-null    object
 4   AREA                    869 non-null    object
 5   DIRECCION HECHO         869 non-null    object
 6   CONTROLES DE TRANSITO   869 non-null    object
 7   BARRIO HECHO            869 non-null    object
 8   CLASE DE ACCIDENTE      869 non-null    object
 9   CLASE DE SERVICIO       867 non-null    object
 10  GRAVEDAD DEL ACCIDENTE  869 non-null    object
 11  CLASE DE VEHICULO       868 non-null    object
 12  Cordenada Geografica    869 non-null    object
dtypes: object(13)
memory usage: 88.4+ KB


In [10]:
## normalizacion de nombres de columnas
df.columns= (
    df.columns
    .str.strip()
    .str.lower()
      .str.replace(" ", "_")
      .str.normalize("NFKD")
      .str.encode("ascii", errors="ignore")
      .str.decode("utf-8")
)

In [11]:
df.columns


Index(['ano', 'fecha', 'dia', 'hora', 'area', 'direccion_hecho',
       'controles_de_transito', 'barrio_hecho', 'clase_de_accidente',
       'clase_de_servicio', 'gravedad_del_accidente', 'clase_de_vehiculo',
       'cordenada_geografica'],
      dtype='object')

In [12]:
## paso de columnas  de fecha de tipo texto a tipo time
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
df[['fecha']].head()


,fecha
0,2023-01-03
1,2023-01-01
2,2023-01-02
3,2023-01-04
4,2023-01-06


In [13]:
## creacion de variables temporales  de fecha para analisis posterior
df['anio'] = df['fecha'].dt.year
df['mes'] = df['fecha'].dt.month
df['dia_semana'] = df['fecha'].dt.day_name()

df[['fecha', 'anio', 'mes', 'dia_semana']].head()


,fecha,anio,mes,dia_semana
0,2023-01-03,2023,1,Tuesday
1,2023-01-01,2023,1,Sunday
2,2023-01-02,2023,1,Monday
3,2023-01-04,2023,1,Wednesday
4,2023-01-06,2023,1,Friday


In [14]:
## normalizacion dias de la semana
map_dias = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

df['dia_semana'] = df['dia_semana'].map(map_dias)
df['dia_semana'].value_counts()


,count
dia_semana,
Domingo,163
Sábado,132
Lunes,124
Jueves,120
Viernes,114
Martes,111
Miércoles,105


In [15]:
## conversion de datos tipo hora a  tiempo
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.time
df[['hora']].head()


,hora
0,15:40:00
1,03:30:00
2,17:30:00
3,09:53:00
4,10:09:00


In [16]:
## Creacion de columna de hora ya limpia
df['hora_num'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.hour
df[['hora', 'hora_num']].head()


,hora,hora_num
0,15:40:00,15.0
1,03:30:00,3.0
2,17:30:00,17.0
3,09:53:00,9.0
4,10:09:00,10.0


In [17]:
## creacion de franjas horarias
def franja_horaria(h):
    if 0 <= h < 6:
        return 'Madrugada'
    elif 6 <= h < 12:
        return 'Mañana'
    elif 12 <= h < 18:
        return 'Tarde'
    else:
        return 'Noche'

df['franja_horaria'] = df['hora_num'].apply(franja_horaria)

df['franja_horaria'].value_counts()


,count
franja_horaria,
Tarde,304
Mañana,246
Noche,233
Madrugada,86


In [18]:
## normalizacion de texto  no informa
def limpiar_texto(col):
    col = col.astype('string')
    col = col.str.strip()
    col = col.str.upper()
    col = col.replace({'NO INFORMA': pd.NA, 'NOINFORMA': pd.NA})
    return col


In [19]:
## limpieza a columnas categóricas
cols_texto = [
    'area',
    'direccion_hecho',
    'controles_de_transito',
    'barrio_hecho',
    'clase_de_accidente',
    'clase_de_servicio',
    'gravedad_del_accidente',
    'clase_de_vehiculo'
]

for c in cols_texto:
    df[c] = limpiar_texto(df[c])

df[cols_texto].head()


,area,direccion_hecho,controles_de_transito,barrio_hecho,clase_de_accidente,clase_de_servicio,gravedad_del_accidente,clase_de_vehiculo
0,URBANA,CARRERA 21 CALLE 32,VERTICAL,SAJONIA,CHOQUE,PARTICULAR,CON HERIDOS,MOTOCICLETA
1,URBANA,CARRERA 25 CALLE 31-19,<NA>,<NA>,CHOQUE,PARTICULAR,CON MUERTO,MOTOCICLETA
2,URBANA,CARERRA 40 CALLE 13C,<NA>,<NA>,CHOQUE,<NA>,CON HERIDOS,CAMION
3,RURAL,LA IBERIA,<NA>,LA MARINA,CHOQUE,PARTICULAR,CON MUERTO,MOTOCICLETA
4,RURAL,VIA FENICIA VEREDA LAS PALMAS,<NA>,LAS PALMAS,VOLCAMIENTO,PARTICULAR,CON MUERTO,AUTOMOVIL


In [20]:
## verificacion de porcentaje de datos sucios  por columnas
df[cols_texto].isna().mean().mul(100).round(2)


,0
area,0.00
direccion_hecho,0.00
controles_de_transito,17.72
barrio_hecho,18.99
clase_de_accidente,0.35
clase_de_servicio,0.81
gravedad_del_accidente,17.84
clase_de_vehiculo,0.46


In [21]:
##
df['mes_nombre'] = df['fecha'].dt.month_name()
df[['mes', 'mes_nombre']].head()


,mes,mes_nombre
0,1,January
1,1,January
2,1,January
3,1,January
4,1,January


In [22]:
##Pasar los meses a español
map_meses = {
    'January': 'Enero',
    'February': 'Febrero',
    'March': 'Marzo',
    'April': 'Abril',
    'May': 'Mayo',
    'June': 'Junio',
    'July': 'Julio',
    'August': 'Agosto',
    'September': 'Septiembre',
    'October': 'Octubre',
    'November': 'Noviembre',
    'December': 'Diciembre'
}

df['mes_nombre'] = df['mes_nombre'].map(map_meses)
df['mes_nombre'].value_counts()


,count
mes_nombre,
Marzo,107
Enero,88
Mayo,86
Febrero,76
Abril,76
Julio,76
Junio,70
Agosto,66
Diciembre,59


In [28]:
##Crear orden del mes
df['mes_orden'] = df['mes']


KeyError: 'mes'

In [32]:
df_final = df[[
    'fecha', 'anio', 'mes', 'mes_nombre', 'mes_orden',
    'dia_semana',
    'hora', 'hora_num', 'franja_horaria',
    'area', 'direccion_hecho', 'barrio_hecho',
    'clase_de_accidente', 'gravedad_del_accidente',
    'clase_de_vehiculo', 'clase_de_servicio',
    'controles_de_transito',
    'coordenada_geografica'
]]

df.shape



KeyError: "None of [Index(['fecha', 'anio', 'mes', 'mes_nombre', 'mes_orden', 'dia_semana', 'hora',\n       'hora_num', 'franja_horaria', 'area', 'direccion_hecho', 'barrio_hecho',\n       'clase_de_accidente', 'gravedad_del_accidente', 'clase_de_vehiculo',\n       'clase_de_servicio', 'controles_de_transito', 'coordenada_geografica'],\n      dtype='object')] are in the [columns]"

In [30]:
df = df.rename(columns={'cordenada_geografica': 'coordenada_geografica'})


In [33]:
df_final.to_csv(
    "accidentalidad_tulua_limpio.csv",
    index=False,
    encoding="utf-8-sig"
)



NameError: name 'df_final' is not defined

In [ ]:
## visual accidentes por dia de la semana
orden_dias = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']
conteo_dias = df['dia_semana'].value_counts().reindex(orden_dias)

plt.figure()
conteo_dias.plot(kind='bar')
plt.title("Accidentes por día de la semana")
plt.xlabel("Día")
plt.ylabel("Número de accidentes")
plt.xticks(rotation=45)
plt.show()


In [ ]:
## accidentes por hora
conteo_hora = df['hora_num'].value_counts().sort_index()

plt.figure()
conteo_hora.plot(kind='bar')
plt.title("Accidentes por hora del día")
plt.xlabel("Hora (0-23)")
plt.ylabel("Número de accidentes")
plt.show()


In [ ]:
##Accidentes por franja horaria
orden_franjas = ['Madrugada','Mañana','Tarde','Noche']
conteo_franja = df['franja_horaria'].value_counts().reindex(orden_franjas)

plt.figure()
conteo_franja.plot(kind='bar')
plt.title("Accidentes por franja horaria")
plt.xlabel("Franja")
plt.ylabel("Número de accidentes")
plt.xticks(rotation=0)
plt.show()


In [ ]:
## Accidentes por area
if 'area' in df.columns:
    conteo_area = df['area'].astype('string').str.strip().str.upper().value_counts()

    plt.figure()
    conteo_area.plot(kind='bar')
    plt.title("Accidentes por área")
    plt.xlabel("Área")
    plt.ylabel("Número de accidentes")
    plt.xticks(rotation=0)
    plt.show()
else:
    print("No existe la columna 'area'")


In [ ]:
## barrios con mas accidentes
if 'barrio_hecho' in df.columns:
    barrios = df['barrio_hecho'].astype('string').str.strip().str.upper()
    barrios = barrios.replace({'NO INFORMA': pd.NA, '': pd.NA})

    top_barrios = barrios.value_counts().head(10)

    plt.figure()
    top_barrios.sort_values().plot(kind='barh')
    plt.title("Top 10 barrios con más accidentes (cuando hay información)")
    plt.xlabel("Número de accidentes")
    plt.ylabel("Barrio")
    plt.show()
else:
    print("No existe la columna 'barrio_hecho'")


***ANALISIS DE ACCIDENTALIDAD AÑO 2024***

In [34]:
##Limpieza mínima del tipo de vehículo
vehiculo = (
    df_2024['clase_de_vehiculo']
    .astype('string')
    .str.strip()
    .str.upper()
)

vehiculo = vehiculo.replace({
    'NO INFORMA': pd.NA,
    '': pd.NA
})


NameError: name 'df_2024' is not defined

In [ ]:
##Conteo de accidentes por tipo de vehículo (2024)
conteo_vehiculos_2024 = vehiculo.value_counts()

conteo_vehiculos_2024


In [ ]:
top_vehiculos_2024 = conteo_vehiculos_2024.head(10)

plt.figure()
top_vehiculos_2024.sort_values().plot(kind='barh')
plt.title("Top 10 tipos de vehículo con más accidentes – 2024")
plt.xlabel("Número de accidentes")
plt.ylabel("Tipo de vehículo")
plt.show()


In [ ]:
tabla_gravedad = (
    df_2024
    .groupby(['clase_de_vehiculo', 'gravedad_del_accidente'])
    .size()
    .unstack(fill_value=0)
)

tabla_gravedad


In [ ]:
tabla_gravedad.loc[
    tabla_gravedad.sum(axis=1).sort_values(ascending=False).head(5).index
].plot(kind='bar', stacked=True)

plt.title("Gravedad del accidente por tipo de vehículo – 2024")
plt.xlabel("Tipo de vehículo")
plt.ylabel("Número de accidentes")
plt.show()


EVOLUCION ACCIDENTALIDDA MES  AMES

In [25]:
# Partimos del df_final que ya tienes
accidentes_mensuales = (
    df_final
    .groupby(['anio', 'mes'])
    .size()
    .reset_index(name='accidentes')
)


NameError: name 'df_final' is not defined